# Fragments

> fragment related functions

In [ ]:
#| default_exp fragments

In [ ]:
#| hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
#| export

from __future__ import annotations
from chem_templates.imports import *
from chem_templates.utils import *
from chem_templates.chem import to_mol, to_smile, canon_smile, Molecule

from rdkit import Chem

/home/dmai/miniconda3/envs/chem_templates/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#| export

def fuse_mol_on_atom_mapping(mol: Chem.Mol) -> Union[Chem.Mol, None]:
    try:
        return Chem.molzip(mol)
    except:
        return None
    
def fuse_smile_on_atom_mapping(smile: str) -> str:
    mol = to_mol(smile)
    mol = fuse_mol_on_atom_mapping(mol)
    if mol is not None:
        return to_smile(mol)
    else:
        return ''

In [ ]:
assert fuse_smile_on_atom_mapping('[*:1]C.[*:1]N') == 'CN'

In [ ]:
#| export

def remove_fragment_mapping(smile: str) -> str:
    patt = re.compile('\[\*(.*?)]')
    smile = patt.sub('[*]', smile)
    return canon_smile(smile)

def add_fragment_mapping(smile:    str, 
                         map_nums: list[int]) -> str:
    
    assert smile.count('*') == len(map_nums)
    
    smile = remove_fragment_mapping(smile)
    
    new_smile = ''
    map_idx = 0
    for char in smile:
        if char=='*':
            new_smile += f'[*:{map_nums[map_idx]}]'
            map_idx += 1
        else:
            new_smile += char
            
    return canon_smile(new_smile)

In [ ]:
assert add_fragment_mapping('*C', [1]) == 'C[*:1]'
assert remove_fragment_mapping('C[*:1]') == '*C'

In [ ]:
#| export

def get_dummy_mol(name:     str, 
                  map_nums: list[int],
                  id:       Optional[int]=None
                 ) -> Chem.Mol:
    templates = {
        0 : '[Zr]',
        1 : '[*][Zr]',
        2 : '[*][Zr][*]',
        3 : '[*][Zr]([*])[*]',
        4 : '[*][Zr]([*])([*])[*]'
    }
    
    num_attachments = len(map_nums)
    mapping_idx = 0
    template = templates[num_attachments]
    template = add_fragment_mapping(template, map_nums)
    
    if id is not None:
        template = template.replace('Zr', f'Zr:{id}')
            
    mol = to_mol(template)
    for atom in mol.GetAtoms():
        if atom.GetAtomicNum() != 0:
            atom.SetProp('atomLabel', name)
            
    return mol

def combine_dummies(dummies: list[Chem.Mol], 
                    fuse:    bool=True) -> Chem.Mol:
    combo = Chem.MolFromSmiles('')
    for mol in dummies:
        combo = Chem.CombineMols(combo, mol)
        
    if fuse:
        combo = Chem.molzip(combo)
        
    return combo

In [ ]:
dummies = [get_dummy_mol('R1', [1]), get_dummy_mol('R2', [1])]
assert [to_smile(i) for i in dummies] == ['[Zr][*:1]', '[Zr][*:1]']
fused = combine_dummies(dummies)
assert to_smile(fused) == '[Zr][Zr]'

In [ ]:
#| export

def is_mapped(smile: str) -> bool:
    patt = re.compile('\[\*(.*?)]')
    return len(patt.findall(smile)) == smile.count('*')

def match_mapping(molecule: Molecule, mapping_idxs: list[int]):
    patt = re.compile('\[\*(.*?)]')
    frag_idxs = [int(i[1:]) for i in patt.findall(molecule.smile)]
    return sorted(frag_idxs) == sorted(mapping_idxs)

In [ ]:
assert not is_mapped('*C')
assert is_mapped('[*:1]C')
assert not is_mapped('[*:1]C*')
assert match_mapping(Molecule('[*:2]C'), [2])
assert not match_mapping(Molecule('[*:2]C'), [1])

In [ ]:
#| export

def generate_mapping_permutations(smile:    str, 
                                  map_nums: list[int], 
                                  exact:    bool=False) -> list[str]:
    
    n_attachments = smile.count('*')
    
    if map_nums is None:
        map_nums = list(range(1, n_attachments+1))
    
    if exact:
        assert n_attachments == len(map_nums)
    else:
        assert n_attachments <= len(map_nums)
    
    perms = permutations(map_nums, n_attachments)
    outputs = []
    for p in perms:
        outputs.append(add_fragment_mapping(smile, p))
        
    return outputs

In [ ]:
assert generate_mapping_permutations('*C*', [2,3,4]) == ['C([*:2])[*:3]',
 'C([*:2])[*:4]',
 'C([*:2])[*:3]',
 'C([*:3])[*:4]',
 'C([*:2])[*:4]',
 'C([*:3])[*:4]']

In [ ]:
#| export

def match_and_map(fragment: str, mapping_idxs: list[int]) -> list[str]:
    outputs = []
    if is_mapped(fragment):
        if match_mapping(Molecule(fragment), mapping_idxs):
            outputs = [fragment]
    elif fragment.count('*')==len(mapping_idxs):
        outputs = generate_mapping_permutations(fragment, mapping_idxs, exact=True)
    
    return outputs

In [ ]:
assert match_and_map('*C*', [1,2]) == ['C([*:1])[*:2]', 'C([*:1])[*:2]']
assert match_and_map('C([*:1])[*:2]', [4,5]) == []
assert match_and_map('C([*:1])[*:2]', [1,2]) == ['C([*:1])[*:2]']

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()